In [145]:
import os 
import sys 
import pandas as pd 
import re
import pickle
import json
import sqlite3
from pprint import pprint
import datetime
import pdb

In [146]:
sys.version

'3.6.9 (default, Oct  8 2020, 12:12:24) \n[GCC 8.4.0]'

In [147]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth

In [148]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Set up 

In [149]:
with open('../credentials/spotify_creds.pkl', 'rb') as hnd:
    credentials = pickle.load(hnd)

In [150]:
os.environ.update(credentials)

In [151]:
scope = "user-read-recently-played"

In [152]:
spotify = spotipy.Spotify(client_credentials_manager= SpotifyOAuth(scope=scope
                                                                   , username='malchemist02'))

## Functions

In [153]:
def split_utc_time_str(time_str):
    if '.' in time_str:
        part1 = datetime.datetime.strptime(time_str, '%Y-%m-%dT%H:%M:%S.%fZ') - datetime.timedelta(hours=4)
    else:
        part1 = datetime.datetime.strptime(time_str, '%Y-%m-%dT%H:%M:%SZ') - datetime.timedelta(hours=4)
    date = str(part1.date())
    time = str(part1.time().strftime('%H:%M'))
    return(date, time)

In [154]:
def get_recently_played(after=None):
    recently_played = spotify.current_user_recently_played(after=after)
    # Describe json 
    print('Getting songs at ', datetime.datetime.now())
    
    if recently_played['cursors'] == None:
        return(pd.DataFrame(columns = ['name', 'artist_name', 'played_at_date', 'played_at_time'
                            , 'duration_min', 'popularity'
                            , 'song_uri', 'artist_id', 'playlist_id']))
    
    before = recently_played['cursors']['before']
    after = recently_played['cursors']['after']
    n_items = len(recently_played['items'])
    print(f'There are {n_items} songs between '
          , datetime.datetime.fromtimestamp(int(before)/1000)
          , '  and  '
          , datetime.datetime.fromtimestamp(int(after)/1000))
    
    if n_items == 0:
        return(pd.DataFrame(columns = ['name', 'artist_name', 'played_at_date', 'played_at_time'
                            , 'duration_min', 'popularity'
                            , 'song_uri', 'artist_id', 'playlist_id']))
    
    # Parse recently played output 
    songs_list = []
    wanted_keys = ['name', 'duration_ms', 'popularity', 'uri']
    for song in recently_played['items']:
        temp_song_dict = {}
        temp_song_dict['after_ts'] = after
        temp_song_dict['played_at_date'], temp_song_dict['played_at_time'] = split_utc_time_str(song['played_at'])
        for k in wanted_keys:
            temp_song_dict[k] = song['track'][k]
            
        # Take first artist information 
        temp_song_dict['artist_name'] = song['track']['artists'][0]['name'] 
        temp_song_dict['artist_id'] = song['track']['artists'][0]['uri']

        if song['context'] != None:
            temp_song_dict['playlist_id'] = song['context']['uri']
        else:
            temp_song_dict['playlist_id'] = None
        songs_list.append(temp_song_dict)

    
    # Export cleanly 
    out_series = pd.DataFrame(songs_list)
    out_series['duration_min'] = out_series['duration_ms']/60000.0
    out_series = out_series.rename({'uri':'song_uri'}, axis=1) 
    out_series = out_series[['name', 'artist_name', 'played_at_date', 'played_at_time'
                            , 'duration_min', 'popularity'
                            , 'song_uri', 'artist_id', 'playlist_id', 'after_ts']]
    
    return(out_series)

## Get time of last Spotify Pull

In [155]:
db_location = 'data/listening_history.db'

In [156]:
con = sqlite3.connect(db_location)
cursor = con.cursor()

In [157]:
try:
    rslt = cursor.execute('select max(after_ts) from Listening_History').fetchone()
    latest_time_pull = rslt[0]
    print("Latest pull from ", 
    datetime.datetime.fromtimestamp(int(latest_time_pull)/1000)) 
except Exception as e:
    # Mainly for first run through when table doesn't exist yet 
    ## Will take the latest recent history available 
    print(e)
    latest_time_pull = None

Latest pull from  2021-01-22 22:15:54.798000


## Pull from Spotify and Store to DB 

In [159]:
newly_played = get_recently_played(latest_time_pull)

Getting songs at  2021-01-22 22:45:31.885004
There are 5 songs between  2021-01-22 22:20:19.177000   and   2021-01-22 22:35:45.667000


In [160]:
newly_played.head()

,name,artist_name,played_at_date,played_at_time,duration_min,popularity,song_uri,artist_id,playlist_id,after_ts
0,La Figura (Como Te Gusta Mami),Gente De Zona,2021-01-22,18:35,4.028217,37,spotify:track:2AeIBdjU4S5KaRqhx4OfIE,spotify:artist:2cy1zPcrFcXAJTP0APWewL,spotify:playlist:5UJhGomjPYIXhtkJiTXMYR,1611354945667
1,Como Volver A Ser Feliz,Luis Enrique,2021-01-22,18:31,4.200433,41,spotify:track:7vItT9jbQYaNAw1o5KVl7N,spotify:artist:2mUI4K6csTQd3jieswcmiI,spotify:playlist:5UJhGomjPYIXhtkJiTXMYR,1611354945667
2,Me Llamaré Tuyo (feat. Gocho) - Remix Version,Víctor Manuelle,2021-01-22,18:27,3.031333,40,spotify:track:6i9ipwB7w8dFwHYz3Mge0C,spotify:artist:4N5fp4zhTsVITZTVfsXpc2,spotify:playlist:5UJhGomjPYIXhtkJiTXMYR,1611354945667
3,Sabes,Luis Enrique,2021-01-22,18:24,4.181333,34,spotify:track:2gxBk9GFuAf18kwIAspriT,spotify:artist:2mUI4K6csTQd3jieswcmiI,spotify:playlist:5UJhGomjPYIXhtkJiTXMYR,1611354945667
4,Mayor Que Yo,Michael Stuart,2021-01-22,18:20,4.406217,35,spotify:track:0pj3zahwh1kZfEVicMhq87,spotify:artist:5iaO1NHFcaQigDtuJi0mhe,spotify:playlist:5UJhGomjPYIXhtkJiTXMYR,1611354945667


In [161]:
con = sqlite3.connect(db_location)
newly_played.to_sql('Listening_History', con, if_exists='append' )

## Get Artist Info 

In [162]:
tables = cursor.execute("SELECT name FROM sqlite_master WHERE type='table'").fetchall()
tables = [x[0] for x in tables]
tables

['Listening_History', 'Artists_Info']

In [163]:
six_months_ago = datetime.datetime.now() - datetime.timedelta(days=6*30)
six_months_ago_str = str(six_months_ago.date())
sql_statement = f'select artist_name, artist_id, pull_date from artists_info where pull_date > {six_months_ago_str}'
sql_statement

'select artist_name, artist_id, pull_date from artists_info where pull_date > 2020-07-26'

In [164]:
con = sqlite3.connect(db_location)

existing_artists = pd.read_sql(sql_statement, con)
existing_artists['uri'] = existing_artists['artist_id']
existing_artists['artist_id'] = 'spotify:artist:' + existing_artists['artist_id']
existing_artists = existing_artists.drop_duplicates(subset=['artist_id'])
existing_artists.sort_values('pull_date', ascending=False).head(10)

,artist_name,artist_id,pull_date,uri
1163,Joshua Bassett,spotify:artist:spotify:artist:4VdV2qRAYBLINR6u...,2021-01-22,spotify:artist:4VdV2qRAYBLINR6uU72V1J
1162,Ricky Montgomery,spotify:artist:spotify:artist:0ZUvK7zGdXLd78mQ...,2021-01-22,spotify:artist:0ZUvK7zGdXLd78mQr3t1Tw
1161,chloe moriondo,spotify:artist:spotify:artist:3P4vW5tzQvmuoNaF...,2021-01-22,spotify:artist:3P4vW5tzQvmuoNaFQqzy9q
1160,Harry Styles,spotify:artist:spotify:artist:6KImCVD70vtIoJWn...,2021-01-22,spotify:artist:6KImCVD70vtIoJWnq6nGn3
1159,Anuel AA,spotify:artist:spotify:artist:2R21vXR83lH98kGe...,2021-01-22,spotify:artist:2R21vXR83lH98kGeO99Y66
1158,BRS Kash,spotify:artist:spotify:artist:5jJjvmEwRr8epuGZ...,2021-01-22,spotify:artist:5jJjvmEwRr8epuGZq4eUUa
1157,Kim Mitzo Thompson,spotify:artist:spotify:artist:258Dm9AVtfbZA1w7...,2021-01-22,spotify:artist:258Dm9AVtfbZA1w7gnEtzq
771,Gene Autry,spotify:artist:spotify:artist:5ixB75BQR3ADoWQk...,2021-01-21,spotify:artist:5ixB75BQR3ADoWQkcHQJTs
778,Willy Chirino,spotify:artist:spotify:artist:4dvonCK12HCv1UUr...,2021-01-21,spotify:artist:4dvonCK12HCv1UUryzRppO
777,Lil Nas X,spotify:artist:spotify:artist:7jVv8c5Fj3E9VhNj...,2021-01-21,spotify:artist:7jVv8c5Fj3E9VhNjxT4snq


In [165]:
existing_artists.shape

(1164, 4)

In [166]:
unique_newly_played = newly_played[['artist_name','artist_id']].drop_duplicates(['artist_id'])
cnt_unique_newly_played = unique_newly_played.shape[0]
merged = pd.merge(unique_newly_played, existing_artists
                  , left_on='artist_id', right_on='uri'
                  , how='left', indicator='True')
new_artists = merged[merged['True'] == 'left_only']
new_artists_ids = new_artists['artist_id_x'].values.tolist()
new_artists

,artist_name_x,artist_id_x,artist_name_y,artist_id_y,pull_date,uri,True


In [167]:
existing_artists[existing_artists['artist_name'].str.contains('Kirst')]

,artist_name,artist_id,pull_date,uri
1155,Kirsty MacColl,spotify:artist:spotify:artist:6UlNIFEuWlBqb4Tv...,2021-01-21,spotify:artist:6UlNIFEuWlBqb4TvlVCekq


In [168]:
def parse_artist_rslt(artist_dict):
    out = {}
    out['artist_name'] = artist_dict.get('name')
    out['uri'] = artist_dict.get('id')
    out['artist_id'] = 'spotify:artist:' + out['uri']
    out['followers'] = artist_dict.get('followers').get('total')
    out['genres'] = str(artist_dict.get('genres'))
    out['popularity'] = artist_dict.get('popularity')
    out['pull_date'] = datetime.datetime.now().date()
    
    out_series = pd.Series(out)
    return(out_series)

In [169]:
if len(new_artists_ids) > 0:
    print("Pulling new artist info")
    artists_rslt = spotify.artists(new_artists_ids)
    artist_series = [parse_artist_rslt(x) for x in artists_rslt['artists']]
    artist_df = pd.concat(artist_series, axis=1).T
#     artist_df = artist_df[['artist_name', 'artist_id', 'followers', 'genres',
#        'popularity', 'pull_date', 'uri']]
    artist_df.to_sql('Artists_Info', con, index=False, if_exists='append')


In [170]:
con.commit()
con.close()

In [171]:
if 'artist_df' in locals():
    print(artist_df)

## Get Playlists